In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [2]:
test_df = pd.read_csv("./data/ratings_test.txt", sep="\t")
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [3]:
import re
from konlpy.tag import Mecab
mecab = Mecab()

In [4]:
def text_clean(x):
    # 한글, 영문대소문자, 숫자만 남기고 모두 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9]', " ", x)
    cleaned = cleaned.replace("  ", " ").replace("  ", " ").strip()
    return cleaned

In [5]:
def tokenizer(text):
    tokens = mecab.morphs(text)
    return tokens

# document 컬럼 전처리(특수문자 제거)

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [7]:
test_df = test_df.dropna()

In [8]:
test_df.loc[:,'document'] = test_df['document'].apply(text_clean)
test_df['document']

0                                                    굳
1                                 GDNTOPCLASSINTHECLUB
2                  뭐야 이 평점들은  나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                            지루하지는 않은데 완전 막장임 돈주고 보기에는
4        3D만 아니었어도 별 다섯 개 줬을텐데 왜 3D로 나와서 제 심기를 불편하게 하죠
                             ...                      
49995          오랜만에 평점 로긴했네 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996     의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997                  그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다
49998       절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네
49999                                       마무리는 또 왜이래
Name: document, Length: 49997, dtype: object

In [9]:
X = test_df['document']
y = test_df['label']

In [10]:
import joblib

In [11]:
cv = joblib.load("./data/cv_mecab.joblib")

In [12]:
tfidf = joblib.load("./data/tfidf_mecab.joblib")

In [13]:
cv_X = cv.transform(X)
tfidf_X = tfidf.transform(X)

# countvectorizer로 훈련한 모델

In [14]:
from sklearn.metrics import classification_report

In [15]:
mnb1 = joblib.load("./data/mnb1_model.joblib")

In [16]:
pred1 = mnb1.predict(cv_X)
print(classification_report(y, pred1))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85     24826
           1       0.86      0.84      0.85     25171

    accuracy                           0.85     49997
   macro avg       0.85      0.85      0.85     49997
weighted avg       0.85      0.85      0.85     49997



In [17]:
mnb2 = joblib.load("./data/mnb2_model.joblib")

In [18]:
pred2 = mnb2.predict(tfidf_X)
print(classification_report(y, pred2))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86     24826
           1       0.87      0.84      0.86     25171

    accuracy                           0.86     49997
   macro avg       0.86      0.86      0.86     49997
weighted avg       0.86      0.86      0.86     49997

